In [1]:
import psycopg2

In [2]:
def deleteFeaturesActionsResults():
    try:
        conn = psycopg2.connect("dbname='ncpfast_dev' user='django_dev' host='ncpfast-db.edc.renci.org' port='5432'")
        cur = conn.cursor()
        
        cur.execute("""SET CLIENT_ENCODING TO UTF8""")
        cur.execute("""SET STANDARD_CONFORMING_STRINGS TO ON""")
        cur.execute("""BEGIN""")

        cur.execute("""DELETE FROM specimens s
                       USING sampling_features sf
                       WHERE s.samplingfeatureid=sf.samplingfeatureid
                         AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM related_features rf 
                       USING sampling_features sf
                       WHERE rf.from_samplingfeature_id=sf.samplingfeatureid
                         AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM sampling_feature_external_identifiers ei 
                       USING sampling_features sf
                       WHERE ei.samplingfeatureid=sf.samplingfeatureid
                         AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM sampling_feature_extension_property_values s
                       USING sampling_features sf
                       WHERE s.samplingfeatureid=sf.samplingfeatureid
                         AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM related_actions ra
                       USING actions a,
                             feature_actions fa,
                             sampling_features sf
                       WHERE ra.actionid=a.actionid
                       AND a.actionid=fa.actionid
                       AND fa.samplingfeatureid=sf.samplingfeatureid
                       AND sf.samplingfeaturecode LIKE 'NC%'""")
        
        cur.execute("""DELETE FROM actions a
                       USING feature_actions fa,
                             sampling_features sf
                       WHERE a.actionid=fa.actionid
                       AND fa.samplingfeatureid=sf.samplingfeatureid
                       AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM measurement_result_values v
                       USING results r,
                             feature_actions fa,
                             sampling_features sf
                       WHERE v.resultid=r.resultid
                       AND r.featureactionid=fa.featureactionid
                       AND fa.samplingfeatureid=sf.samplingfeatureid
                       AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM results r
                       USING feature_actions fa,
                             sampling_features sf
                       WHERE r.featureactionid=fa.featureactionid
                       AND fa.samplingfeatureid=sf.samplingfeatureid
                       AND sf.samplingfeaturecode LIKE 'NC%'""")
        
        cur.execute("""DELETE FROM feature_actions fa
                       USING sampling_features sf
                       WHERE fa.samplingfeatureid=sf.samplingfeatureid
                       AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM sites s
                       USING sampling_features sf
                       WHERE s.samplingfeatureid=sf.samplingfeatureid
                       AND sf.samplingfeaturecode LIKE 'NC%'""")

        cur.execute("""DELETE FROM sampling_features sf WHERE sf.samplingfeaturecode LIKE 'NC%'""")

        conn.commit()
        cur.execute("""ANALYZE specimens""")
        cur.execute("""ANALYZE related_features""")
        cur.execute("""ANALYZE sampling_feature_external_identifiers""")
        cur.execute("""ANALYZE sampling_feature_extension_property_values""")
        cur.execute("""ANALYZE sites""")
        cur.execute("""ANALYZE sampling_features""")

    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        if cur is not None:
            cur.close()
        if conn is not None:
            conn.close() 
            

In [3]:
deleteFeaturesActionsResults()